In [1]:
# 1. Import libraries
import pandas as pd

# 2. Load the dataset
df = pd.read_csv("processed_data.csv", low_memory=False)

# 3. Calculate percentage share of kilometers per country
km_columns = ['AT_KM', 'BE_KM', 'CZ_KM', 'DE_KM', 'DK_KM', 'EE_KM', 'ES_KM', 
              'FI_KM', 'HR_KM', 'FR_KM', 'HU_KM', 'IT_KM', 'LT_KM', 'LV_KM',
              'NL_KM', 'PL_KM', 'RO_KM', 'SE_KM', 'SI_KM', 'SK_KM']

for col in km_columns:
    country_code = col.replace('_KM', '')
    df[f'{country_code}_KM_PERC'] = (df[col] / df['TOTAL_KM']) * 100

# 4. Calculate group mean features for selected categorical columns
categorical_columns = ['COD_LP', 'COD_DP', 'ROUTE_TYPE', 'VEHICLE_TYPE', 
                       'BODY_TYPE', 'LOAD_UNLOAD_METHOD', 'GOODS_TYPE', 
                       'CARGO_TYPE', 'DOCUMENTS_BY']

for col in categorical_columns:
    df[f'{col}_MEAN_PRICE_PER_KM'] = df.groupby(col)['PRICE_PER_KM'].transform('mean')

# 5. Convert TIME_OF_ENTRY to datetime and extract features
df['TIME_OF_ENTRY'] = pd.to_datetime(df['TIME_OF_ENTRY'], errors='coerce')

df['ENTRY_DAY'] = df['TIME_OF_ENTRY'].dt.day
df['ENTRY_WEEKDAY'] = df['TIME_OF_ENTRY'].dt.weekday
df['ENTRY_MONTH'] = df['TIME_OF_ENTRY'].dt.month
df['ENTRY_YEAR'] = df['TIME_OF_ENTRY'].dt.year
df['ENTRY_DAYOFYEAR'] = df['TIME_OF_ENTRY'].dt.dayofyear

# 6. Calculate mean PRICE_PER_KM for each extracted time feature
time_features = ['ENTRY_DAY', 'ENTRY_WEEKDAY', 'ENTRY_MONTH', 'ENTRY_YEAR', 'ENTRY_DAYOFYEAR']

for feature in time_features:
    df[f'{feature}_MEAN_PRICE_PER_KM'] = df.groupby(feature)['PRICE_PER_KM'].transform('mean')

# 7. Correctly impute missing values
df['TEMP_MIN'] = df['TEMP_MIN'].fillna(df['TEMP_MIN'].min())
df['TEMP_MAX'] = df['TEMP_MAX'].fillna(df['TEMP_MAX'].max())
df['PAYMENT_TERM'] = df['PAYMENT_TERM'].fillna(df['PAYMENT_TERM'].mean())

# 8. Round time features to the nearest hour with specified format
time_columns = ['START_LOAD_TIME', 'END_LOAD_TIME', 'START_DELIVERY_TIME', 'END_DELIVERY_TIME']
for col in time_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce', format='%H:%M:%S').dt.round('h')

# 9. Replace time features with mean PRICE_PER_KM per rounded hour
for col in time_columns:
    df[col+'_MEAN_PRICE_PER_KM'] = df.groupby(df[col].dt.hour)['PRICE_PER_KM'].transform('mean')

# 10. Drop unnecessary columns
columns_to_drop = km_columns + categorical_columns + time_columns + [
    'START_LOAD_DATE', 'END_LOAD_DATE',
    'START_DELIVERY_DATE', 'END_DELIVERY_DATE',
    'START_LOAD_DATE_YEAR', 'START_LOAD_DATE_MONTH', 'START_LOAD_DATE_WEEK', 'START_LOAD_DATE_WEEKDAY',
    'START_LOAD_DATE_DAY', 'START_LOAD_DATE_DAYOFYEAR', 'END_LOAD_DATE_YEAR', 'END_LOAD_DATE_MONTH',
    'END_LOAD_DATE_WEEK', 'END_LOAD_DATE_WEEKDAY', 'END_LOAD_DATE_DAY', 'END_LOAD_DATE_DAYOFYEAR',
    'START_DELIVERY_DATE_YEAR', 'START_DELIVERY_DATE_MONTH', 'START_DELIVERY_DATE_WEEK', 
    'START_DELIVERY_DATE_WEEKDAY', 'START_DELIVERY_DATE_DAY', 'START_DELIVERY_DATE_DAYOFYEAR',
    'END_DELIVERY_DATE_YEAR', 'END_DELIVERY_DATE_MONTH', 'END_DELIVERY_DATE_WEEK',
    'END_DELIVERY_DATE_WEEKDAY', 'END_DELIVERY_DATE_DAY', 'END_DELIVERY_DATE_DAYOFYEAR',
    'TIME_OF_ENTRY', 'LOAD_COUNTRY', 'DELIVERY_COUNTRY', 'RELATION'
]

df.drop(columns=columns_to_drop, inplace=True)

# 11. Impute remaining missing values with mean
for col in df.columns:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mean())

# 12. Save the final dataset
df.to_csv("final_processed_data.csv", index=False)

# 13. Verify final dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45569 entries, 0 to 45568
Data columns (total 60 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   EPALE                                  45569 non-null  int64  
 1   TEMP_MIN                               45569 non-null  float64
 2   TEMP_MAX                               45569 non-null  float64
 3   EUR                                    45569 non-null  float64
 4   LDM                                    45569 non-null  float64
 5   M3                                     45569 non-null  float64
 6   HEIGHT                                 45569 non-null  float64
 7   WIDTH                                  45569 non-null  float64
 8   TONS                                   45569 non-null  float64
 9   OTHER_COSTS                            45569 non-null  float64
 10  QTY_LOADS                              45569 non-null  int64  
 11  QT